## Notations
- B[1-9] : Bambous
- C[1-9] : Cracks
- D[1-9] : Dots
- G      : Green dragon
- R      : Red dragon
- 0      : White dragon
- N      : North wind
- E      : Est wind
- S      : South wind
- W      : West wind
- F      : Flower
- J      : Joker

## conventions d'écriture
- a()    : ensemble couleur 1
- b()    : ensemble couleur 2
- c()    : ensemble couleur 3
- d      : n'importe quel dragon
- <2468> : n'importe quel de ces chiffres, mais tous les groupes dans la mains doivent avoir le même nombre

In [ ]:
#Mains
mains = [
    #2023
    "a(222) 0000 b(222) b(3333)",
    "FF a(2023) b(2222) c(3333)",
    "FFFF a(ddd) b(2023) c(ddd)",
    "a(22) 000 NEWS b(222) b(33)",

    #2468
    "FFFF a(2222) a(46) a(8888)",
    "a(22) a(46) a(88) b(<2468><2468><2468><2468>) c(<2468><2468><2468><2468>)",
    "a(222) a(4444) a(666) a(8888)",
    "a(222) a(4444) b(666) b(8888)",
    "a(22) a(4444) b(44) b(6666) c(88)",
    "a(222) a(888) b(dddd) c(dddd)",
    "FF a(222) a(44) a(66) a(888) a(dd)",

    #ANY LIKE NUMBERS
    "FFFF a(<123456789><123456789><123456789><123456789>) b(<123456789><123456789>) c(<123456789><123456789><123456789><123456789>)",
    "FF a(<123456789><123456789><123456789>) a(d) b(<123456789><123456789><123456789>) b(d) c(<123456789><123456789><123456789>) c(d)",

    #ADDITION HANDS
    "FF a(1111) b(1111) c(2222)",
    "FF a(2222) b(2222) c(4444)",
    "FF a(3333) b(3333) c(6666)",
    "FF a(4444) b(4444) c(8888)",

    #QUINTS
    "a(1111J) (<NWSE><NWSE><NWSE><NWSE>) b(1111J)"

]

In [ ]:

import re
from itertools import permutations
import pandas as pd

def analyseMain(main, id_main):
    main = main.translate(str.maketrans('', '', '()'))
    ensembles = main.split()

    structure = {
        'a' : [],   #Couleur 1
        'b' : [],   #Couleur 2
        'c' : [],   #Couleur 3
        'e' : [],    #Autres
        'g' : [],    #Groupes numériques (comme <2468>)
        'ge': []    #Groupes non numériques (comme <NEWS>)
    }

    nbCouleurs = 0
    couleurs = ['B', 'C', 'D']

    for ensemble in ensembles:
        if ensemble[0] == '<':
            #Cas d'un groupe non colorisé
            mains, g = decoupeEnsemble(ensemble[0:], structure)
            structure['e'].append(mains)
            # Modifier ici
        elif ensemble[0] in ['a','b','c']:
            if len(structure[ensemble[0]]) == 0:
                nbCouleurs = nbCouleurs +1
            mains, g = decoupeEnsemble(ensemble[1:], structure)
            structure[ensemble[0]].append(mains)
            structure['g'] = g
        else:
            mains, g = decoupeEnsemble(ensemble, structure)
            structure['e'].append(mains)
            structure['g'] = g
    
    print(structure)

    combinaisons = list(permutations(couleurs, nbCouleurs))
    mainsPossibles = []
    for combinaison in combinaisons:
        mainsPossibles = mainsPossibles + calculerMainsDerivees(structure, combinaison, nbCouleurs)

    print(mainsPossibles)
    mainsPossibles = pd.DataFrame(mainsPossibles, columns=['1','2','3','4','5','6','7','8','9','10','11','12','13','14'])

    #Suppression des mains en doubles
    # Créer une colonne temporaire avec les valeurs triées
    mainsPossibles['sorted_values'] = mainsPossibles.iloc[:, 1:15].apply(lambda row: tuple(sorted(row)), axis=1)

    # Supprimer les doublons basés sur cette colonne + id_main
    df_unique = mainsPossibles.drop_duplicates(subset=['sorted_values'], keep='first')

    # Supprimer la colonne temporaire
    mainsPossibles = df_unique.drop('sorted_values', axis=1)

    mainsPossibles['id_main'] = id_main



    return mainsPossibles


def decoupeEnsemble(ensemble, structure):
    tiles = []
    groupes = structure['g']

    #Cas [2468]
    if ensemble[0] == '<':
        segments = ensemble.split("<")
        for s in segments:
            if len(s) > 1:
                groupName = f"<{s}"
                if groupName not in groupes:
                    groupes.append(groupName)
        
        for g in groupes:
            count = ensemble.count(g)
            if count > 0:
                tiles.append(f"{g}*{count}")
    else:
        for c in ensemble:
            tiles.append(c)

    return tiles, groupes

def calculerMainsDerivees(structure, combinaison, nbCouleurs):
    mainsDerivees = []
    print(f"Calcul combinaison {combinaison} avec {nbCouleurs} couleurs")

    if len(structure['g']) > 0:
        #On a des groupes spécifiques
        print("Traitement groupe spécifique")
        for g in structure['g']:
            #On récupère les valeurs possibles dans le groupe
            values = g.translate(str.maketrans('', '', '<>'))
            for v in values:
                mainsDerivees = mainsDerivees + [calculerMainsDeriveesSegment(structure, combinaison, nbCouleurs, g, v)]
    else:
        #Pas de groupes spécifiques
        mainsDerivees = mainsDerivees + [calculerMainsDeriveesSegment(structure, combinaison, nbCouleurs)]

    return mainsDerivees

def calculerMainsDeriveesSegment(structure, combinaison, nbCouleurs, groupeSpecifique=None, valeurGroupe=None):
    main = []

    groupesCouleur = ['a', 'b', 'c']
    combinaisonDragons = {
        'B' : 'G',
        'C' : 'R',
        'D' : '0'
    }    

    for gc in range(0, nbCouleurs):
        couleur = groupesCouleur[gc]
        for groupe in structure[couleur]:
            for tile in groupe:
                if tile == "J":
                    #Cas du joker
                    main.append("J")
                elif tile[0] == '<':
                    #Cas d'un groupe spécifique
                    count = int(tile.split("*")[1])
                    nomGroupe = tile.split("*")[0]
                    if nomGroupe == groupeSpecifique:
                        for i in range(0, count):
                            main.append(f"{combinaison[gc]}{valeurGroupe}")
                elif tile == 'd':
                    #Cas de dragons (non déterminés)
                    main.append(f"{combinaisonDragons[combinaison[gc]]}")
                elif tile == '0':
                    #Cas du dragon blanc
                    main.append(f"{tile}")
                else:
                    #Autre cas
                    main.append(f"{combinaison[gc]}{tile}")
    for groupe in structure['e']:
        for tile in groupe:
            main.append(tile)

    return main








In [44]:
out = analyseMain(mains[17], 5)
out


{'a': [['1', '1', '1', '1', 'J']], 'b': [['1', '1', '1', '1', 'J']], 'c': [], 'e': [['<NWSE>*4']], 'g': ['<NWSE>']}
Calcul combinaison ('B', 'C') avec 2 couleurs
Traitement groupe spécifique
Calcul combinaison ('B', 'D') avec 2 couleurs
Traitement groupe spécifique
Calcul combinaison ('C', 'B') avec 2 couleurs
Traitement groupe spécifique
Calcul combinaison ('C', 'D') avec 2 couleurs
Traitement groupe spécifique
Calcul combinaison ('D', 'B') avec 2 couleurs
Traitement groupe spécifique
Calcul combinaison ('D', 'C') avec 2 couleurs
Traitement groupe spécifique
[['B1', 'B1', 'B1', 'B1', 'J', 'C1', 'C1', 'C1', 'C1', 'J', '<NWSE>*4'], ['B1', 'B1', 'B1', 'B1', 'J', 'C1', 'C1', 'C1', 'C1', 'J', '<NWSE>*4'], ['B1', 'B1', 'B1', 'B1', 'J', 'C1', 'C1', 'C1', 'C1', 'J', '<NWSE>*4'], ['B1', 'B1', 'B1', 'B1', 'J', 'C1', 'C1', 'C1', 'C1', 'J', '<NWSE>*4'], ['B1', 'B1', 'B1', 'B1', 'J', 'D1', 'D1', 'D1', 'D1', 'J', '<NWSE>*4'], ['B1', 'B1', 'B1', 'B1', 'J', 'D1', 'D1', 'D1', 'D1', 'J', '<NWSE>*4'], [

ValueError: 14 columns passed, passed data had 11 columns